In [1]:
%reload_ext ishbook
import pandas as pd
import numpy as np
import datetime as dt
import time
import iql
import plus
import re
from df2gspread import df2gspread as d2g
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
cs_team_list = plus.read_google_sheet('1194g28N02sqVJMhHHQthdKItgzgbwWwaAfgC5YVVFko', intype ='key', header=True, sheet = 'Current Team Info', index = False)

LDAP name:
dmoncada
Password:
········


In [3]:
cs_team_list.columns

Index([u'', u'First Name', u'Last Name', u'Common Name', u'Payroll Name',
       u'Team', u'Role', u'Manager', u'Location', u'AdCentral Name',
       u'CS Start Date', u'Employee ID', u'AdCentral ID', u'Category',
       u'Email', u'University', u'Username', u'Indeed.com',
       u'Notes/Transition Date (if applicable)', u'Notes Name',
       u'Manager Email', u'Managers' Manager', u'Level 3 Management',
       u'Concatenate', u'Director', u'Ad Central Name'],
      dtype='object')

In [4]:
cs_comparison = cs_team_list[["AdCentral ID","Payroll Name","Manager","Team"]]

In [5]:
%ish workday_team_info = from indeedemployeesnapshot yesterday today \
where division IN (Client Support, Client Services, Small Business) \
group by id/*AdCentral ID*/, full_name, manager_ldap, cost_center, division, team

Output = None

In [6]:
workday_team_info.drop("", axis = 1, inplace = True)

In [7]:
cs_comparison.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 724 entries, 0 to 723
Data columns (total 4 columns):
AdCentral ID    724 non-null object
Payroll Name    724 non-null object
Manager         724 non-null object
Team            724 non-null object
dtypes: object(4)
memory usage: 22.7+ KB


In [8]:
workday_team_info["AdCentral ID"] = workday_team_info["AdCentral ID"].astype(str)

In [9]:
workday_team_list_compare = workday_team_info.merge(cs_comparison, how = "left")

In [11]:
workday_team_list_compare.to_clipboard()

In [37]:
#workday_team_list_compare[pd.isnull(workday_team_list_compare["Team"])]

,AdCentral ID,full_name,manager_ldap,cost_center,division,team,,Payroll Name,Manager,Team
48,3799,Lukas Remus,sascha,C Support - EMEA - Germany - Direct,Client Support,CS-EMEA-DACH,1,NaN,NaN,NaN
110,-1,Kieran Keegan,stevenh,C Support - EMEA - UK Mgmt,Client Support,CS-EMEA-UK,1,NaN,NaN,NaN
380,-1,Maya Yamamoto,kanako,C Support - Japan - Direct,Client Support,CSJapan,1,NaN,NaN,NaN
473,-1,Zachary Sullivan,jtomas,Small Business - Interns,Small Business,CSSmallBusiness,1,NaN,NaN,NaN
474,3746,Ian Moraes,gianna,Small Business - Interns,Small Business,CSSmallBusiness,1,NaN,NaN,NaN
478,5372,Karyn Ryan,jtomas,Small Business - Interns,Small Business,CSSmallBusiness,1,NaN,NaN,NaN
654,-1,Ailish McGovern,emil,C Support - Interns,Client Support,NONE,1,NaN,NaN,NaN
655,-1,Kara Sheehan,margaux,C Support - Interns,Client Support,NONE,1,NaN,NaN,NaN
656,-1,Lindsay Green,jeremy,C Support - Interns,Client Support,NONE,1,NaN,NaN,NaN
657,-1,Niamh O'Gorman,samanthafodor,C Support - Interns,Client Support,NONE,1,NaN,NaN,NaN


In [12]:
d2g.upload(workday_team_list_compare, "1_fKdxkajOWnHf9z20CT9T_pf7jcqohFB9fGeH8b1G68", wks_name = "Sheet2")

<Worksheet 'Sheet2' id:ot6pi9u>

In [20]:
cs_ids = tuple(cs_team_list['Username'].astype(str).tolist())

In [30]:
cs_ids[0:5]

('acrawford', 'adoran', 'abe', 'ajacobs', 'adamk')

In [26]:
print len(cs_ids)

789


In [46]:
%%ish

workday_teams = from indeedemployeesnapshot yesterday today WHERE ldap in {cs_ids} GROUP BY ldap, full_name, team, primary_location, cost_center

Output = None

In [52]:
HTML(workday_teams.to_html())

,ldap,full_name,team,primary_location,cost_center,
0,aemery,Andrea Emery,CSAccountManagement,Stamford,Account Management - Direct,1
1,aleach,Alyssa Leach,CSAccountManagement,Stamford,Account Management - Direct,1
2,alison,Alison Godsill,CSAccountManagement,Stamford,Account Management - Direct,1
3,alli,Allison Andrews,CSAccountManagement,New York City,Account Management - Direct,1
4,amandat,Amanda Thomas,CSAccountManagement,New York City,Account Management - Direct,1
5,amauris,Amauris Maldonado,CSAccountManagement,New York City,Account Management - Direct,1
6,amiller,Aimee Miller,CSAccountManagement,New York City,Account Management - Direct,1
7,amys,Amy Singer,CSAccountManagement,New York City,Account Management - Direct,1
8,andreasm,Andreas Mandelenis,CSAccountManagement,San Mateo,Account Management - Direct,1
9,arothstein,Adam Rothstein,CSAccountManagement,New York City,Account Management - Direct,1


In [33]:
cs_team_list.columns

Index([u'First Name', u'Last Name', u'Common Name', u'Payroll Name', u'Team',
       u'Role', u'Manager', u'Location', u'AdCentral Name', u'CS Start Date',
       u'Employee ID', u'AdCentral ID', u'Category', u'Email', u'University',
       u'Username', u'Indeed.com', u'Notes/Transition Date (if applicable)',
       u'Notes Name', u'Manager Email', u'Managers' Manager',
       u'Level 3 Management', u'Concatenate', u'Director', u'Ad Central Name'],
      dtype='object')

In [39]:
columns = ['First Name', 'Last Name', 'Team', 'Location', 'Username']

In [45]:
cs_team_list[columns].merge(workday_teams, how = 'left', left_on = 'Username', right_on = 'ldap' ).to_csv('Workday vs CS Teamt List.csv', encoding = 'utf-8', index = False)